In [ ]:
!pip install selenium

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from urllib.parse import quote_plus
import re
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from urllib.request import urlretrieve

from selenium import webdriver
import time

In [ ]:
SCROLL_PAUSE_TIMER = 2

In [ ]:
# 네이버 크롤링
def crawling_from_naver(keyword):
    base_url='https://search.naver.com/search.naver?where=image&sm=tab_jum&query='
    url = base_url + keyword
    
    print(f'네이버에서 {keyword} 이미지를 크롤링합니다.')
    
    # driver = webdriver.Chrome('/content/chromedriver.exe')
    from google.colab import files 
    driver = files.upload()
    driver.get(url)
    
    last_height = driver.execute_script('return document.body.scrollHeight')
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(SCROLL_PAUSE_TIMER)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight - 50);')
        time.sleep(SCROLL_PAUSE_TIMER)
        
        new_height = driver.execute_script('return document.body.scrollHeight')
        
        if last_height == new_height:
            more_img_btn = driver.find_elements_by_css_selector('#_sau_imageTab > div > div.more_img > a')[0]
            if more_img_btn.is_displayed():
                more_img_btn.click()
            else:
                break
                
        last_height = new_height
    
    html = driver.page_source
    soup = BeautifulSoup(html)
    imgs = soup.select('img._img')
    
    imgs_list = []
    
    for img in imgs:
        imgs_list.append(img['src'])
    
    saveImages('Naver', keyword, imgs_list)
    
    driver.close()

In [ ]:
def crawling_from_google(keyword):
    base_url = 'https://google.com/search?q='
    url = base_url + quote_plus(keyword)
    
    print(f'구글에서 {keyword} 이미지를 크롤링합니다.')

    # driver = webdriver.Chrome('/content/chromedriver.exe')
    from google.colab import files 
    driver = files.upload()
    driver.get(url)
    
    img_btn = driver.find_elements_by_xpath('//*[@id="hdtb-msb-vis"]/div[2]/a')[0]
    img_btn.click()
    
    last_height = driver.execute_script('return document.body.scrollHeight')
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(SCROLL_PAUSE_TIMER)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight - 50);')
        time.sleep(SCROLL_PAUSE_TIMER)
        
        new_height = driver.execute_script('return document.body.scrollHeight')
        
        if new_height == last_height:
            more_img_btn = driver.find_elements_by_xpath('//*[@id="islmp"]/div/div/div/div/div[5]/input')[0]
            if more_img_btn.is_displayed():
                more_img_btn.click()
            else:
                break
        
        last_height = new_height
        
    img_cnt = len(driver.find_elements_by_tag_name('img'))
    
    if not os.path.exists(f'/Volumes/SeaGate/Mac/data-science/faceImage/Google_{keyword}'):
        os.makedirs(f'/Volumes/SeaGate/Mac/data-science/faceImage/Google_{keyword}')
    
    for i in range(img_cnt):
        img = driver.find_elements_by_tag_name('img')[i]
        img.screenshot(f'/Volumes/SeaGate/Mac/data-science/faceImage/Google_{keyword}/{i}.png')
        print('=', end='')
    
    print('')
    print(f'{keyword} 이미지 크롤링이 완료되었습니다')
    
    driver.close()

In [ ]:
def crawling_from_daum(keyword):
    base_url = 'https://search.daum.net/search?w=img&nil_search=btn&DA=NTB&enc=utf8&q='
    url = base_url + keyword
    
    print(f'다음에서 {keyword} 이미지를 크롤링합니다.')
    
    # driver = webdriver.Chrome('/content/chromedriver.exe')
    from google.colab import files 
    driver = files.upload()
    driver.get(url)
    
    last_height = driver.execute_script('return document.body.scrollHeight')
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        time.sleep(SCROLL_PAUSE_TIMER)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight - 50)')
        time.sleep(SCROLL_PAUSE_TIMER)
        
        new_height = driver.execute_script('return document.body.scrollHeight')
        
        if new_height == last_height:
            more_img_btn = driver.find_elements_by_xpath('//*[@id="imgColl"]/div[5]/a[1]')[0]
            if more_img_btn.is_displayed():
                more_img_btn.click()
            else:
                break
            
        last_height = new_height
    
    html = driver.page_source
    soup = BeautifulSoup(html)
    imgs = soup.select('img.thumb_img')
    imgs_list = []
    
    for img in imgs:
        imgs_list.append(img['src'])
        
    saveImages('Daum', keyword, imgs_list)
    
    driver.close()

In [ ]:
def saveImages(where, keyword, imgs):
    if not os.path.exists(f"D:\data-science\driver\face/{where}_{keyword}"):
        os.makedirs(f"D:\data-science\driver\face/{where}_{keyword}")

    for index, img in enumerate(imgs):
        target = f'D:\data-science\driver\face/{where}_{keyword}/{index}.png'
        urlretrieve(img, target)
        
        print('=', end='')
        
    print('')
    print(f'{keyword} 이미지 크롤링이 완료되었습니다.')

In [ ]:
# keyword = input('검색어를 입력하세요: ')

# crawling_from_daum(keyword)
# crawling_from_google(keyword)
# crawling_from_naver(keyword)


import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')
sys.path.insert(0, '/usr/bin/chromedriver')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--single-process")
chrome_options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('/usr/bin/chromedriver', options=chrome_options)

WebDriverException: ignored